In [ ]:
import cartopy.crs as ccrs
from cmocean import cm 
from dino import Experiment
from matplotlib import colors
from matplotlib import pyplot as plt
import numpy as np
import xarray as xr
import cftime as cft
import xnemogcm as xn
import xgcm

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
dino_exp = Experiment(path, 'GetEmP')

In [ ]:
emp         = (dino_exp.data.saltflx / dino_exp.data.sss)
emp_mean    = dino_exp.grid.average(emp, ['X', 'Y'])

In [ ]:
emp_norm= emp - emp_mean

In [ ]:
dino_exp.grid.average(emp_norm, ['X', 'Y']).plot()

In [ ]:
emp_star = dino_exp.get_emp(d=20)

In [ ]:
emp_star_mean = dino_exp.grid.average(emp_star, ['X', 'Y'])

In [ ]:
emp_star_mean.plot()

In [ ]:
emp_star - emp

In [ ]:
fig, axs = plt.subplots(1,1,figsize=(10,6))
abs(emp_star - emp).max('t_d').plot(x='glamt', y='gphit', ax=axs, cmap=cm.tempo)

In [ ]:
fig, axs = plt.subplots(1,2,figsize=(15,6))
emp_star.mean('t_d').plot(x='glamt', y='gphit', ax=axs[0])
emp_star.var('t_d').plot(x='glamt', y='gphit', ax=axs[1], norm=colors.LogNorm(), cmap=cm.haline)

In [ ]:
emp_norm.isel(t_d=-1).plot()

## Climatology of EmP field from the last ten years

In [ ]:
climatology = emp_star.groupby('t_d.dayofyear').mean()

In [ ]:
climatology_mean = dino_exp.grid.average(climatology, ['X', 'Y'])

In [ ]:
climatology_mean.plot()

## Seasonal variance of the climatology

In [ ]:
variance = climatology.var('dayofyear')
variance.plot(x='glamt', y='gphit',norm=colors.LogNorm(), cmap=cm.haline)

## Maximum seasonal cycle

In [ ]:
idx_y = variance.argmax(...)['y_c'].values
idx_x = variance.argmax(...)['x_c'].values

In [ ]:
climatology.isel(y_c=idx_y, x_c=idx_x).plot()

In [ ]:
climatology = climatology.fillna(0.)
climatology = climatology.rename('emp')

In [ ]:
emp_ds = climatology.to_dataset().drop_vars([
    'x_c',
    'y_c',
    'gphit',
    'glamt',
    'dayofyear',
]).rename({'x_c':'x', 'y_c':'y', 'dayofyear':'t'})

In [ ]:
emp_ds.transpose('t', 'x', 'y').to_netcdf('emp_monthly_txy.nc')
emp_ds.transpose('x', 'y', 't').to_netcdf('emp_monthly_xyt.nc')
emp_ds.transpose('y', 'x', 't').to_netcdf('emp_monthly_yxt.nc')

In [ ]:
emp_ds.to_netcdf('emp_climatology.nc', encoding={'emp': {'_FillValue': False}})

In [ ]:
path   = "/data/dkamm/nemo_output/DINO/"
test_exp = Experiment(path, 'testEmPfield2')

In [ ]:
test_exp.data.empmr.isel(t_d=-1).plot()

In [ ]:
dino_exp